In [2]:
%pip install prophet

  Using cached prophet-1.3.0-py3-none-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached cmdstanpy-1.3.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-macosx_11_0_arm64.whl.metadata (52 kB)
  Using cached holidays-0.91-py3-none-any.whl.metadata (50 kB)
  Using cached tqdm-4.67.3-py3-none-any.whl.metadata (57 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached contourpy-1.3.3-cp314-cp314-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp314-cp314-macosx_10_15_universal2.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp314-cp314-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-12.1.1-cp314-cp314-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached pyparsing-3.3.2-py3-none-any.whl.metadata (5.8 kB)
Using cached prophet-1.3.0-py3-none-macos

In [3]:
import sys
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append('../src')
from forecasting import TradeForecaster

df = pd.read_csv('../data/processed/trade_data_global.csv')
tariff_master = pd.read_csv('../data/reference/trade_war_master.csv')

forecaster = TradeForecaster(df)

countries_to_forecast = ['China', 'Viet Nam', 'India', 'Mexico']

print("🚀 Running Prophet forecasts (takes ~60 seconds)...")

all_forecasts = forecaster.forecast_all_countries(
    countries_to_forecast, 
    metric='exports',
    periods=3 
)

all_forecasts = forecaster.calculate_tariff_impact_on_forecast(
    all_forecasts, tariff_master
)

all_forecasts.to_csv('../data/processed/trade_forecasts.csv', index=False)
print(f"Forecasts complete: {all_forecasts.shape}")

display(all_forecasts[['country_name', 'year', 'actual', 
                       'yhat', 'yhat_tariff_adjusted', 
                       'is_forecast']].tail(10))

19:05:28 - cmdstanpy - INFO - Chain [1] start processing


🚀 Running Prophet forecasts (takes ~60 seconds)...
Forecasting China...


19:05:31 - cmdstanpy - INFO - Chain [1] done processing
19:05:31 - cmdstanpy - INFO - Chain [1] start processing
19:05:31 - cmdstanpy - INFO - Chain [1] done processing
19:05:31 - cmdstanpy - INFO - Chain [1] start processing
19:05:31 - cmdstanpy - INFO - Chain [1] done processing
19:05:31 - cmdstanpy - INFO - Chain [1] start processing
19:05:31 - cmdstanpy - INFO - Chain [1] done processing


Forecasting Viet Nam...
Forecasting India...
Forecasting Mexico...
Forecasts complete: (52, 14)


,country_name,year,actual,yhat,yhat_tariff_adjusted,is_forecast
42,Mexico,2018,4.905090e+11,4.620290e+11,4.620290e+11,False
43,Mexico,2019,5.024676e+11,4.789577e+11,4.789577e+11,False
44,Mexico,2020,4.398539e+11,4.958864e+11,4.958864e+11,False
45,Mexico,2021,5.341216e+11,5.452467e+11,5.452467e+11,False
46,Mexico,2022,6.297887e+11,5.944720e+11,5.944720e+11,False
47,Mexico,2023,6.486423e+11,6.436915e+11,6.436915e+11,False
48,Mexico,2024,6.813487e+11,6.929109e+11,6.929109e+11,False
49,Mexico,2024,6.813487e+11,7.421303e+11,7.421303e+11,False
50,Mexico,2025,NaN,7.913498e+11,7.676093e+11,True
51,Mexico,2026,NaN,8.405692e+11,8.153521e+11,True


In [4]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=countries_to_forecast,
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

positions = [(1,1), (1,2), (2,1), (2,2)]

for idx, country in enumerate(countries_to_forecast):
    row, col = positions[idx]

    country_data = all_forecasts[all_forecasts['country_name'] == country]
    historical = country_data[~country_data['is_forecast']]
    future = country_data[country_data['is_forecast']]

    # Historical actual line
    fig.add_trace(go.Scatter(
        x=historical['year'],
        y=historical['actual'],
        name='Historical' if idx == 0 else None,
        showlegend=(idx == 0),
        mode='lines+markers',
        line=dict(color='#1f77b4', width=2),
        marker=dict(size=6)
    ), row=row, col=col)

     # Confidence interval (shaded)
    fig.add_trace(go.Scatter(
        x=list(future['year']) + list(future['year'])[::-1],
        y=list(future['yhat_upper']) + list(future['yhat_lower'])[::-1],
        fill='toself',
        fillcolor='rgba(31, 119, 180, 0.15)',
        line=dict(color='rgba(255,255,255,0)'),
        name='80% Confidence' if idx == 0 else None,
        showlegend=(idx == 0)
    ), row=row, col=col)

    # Baseline forecast line
    fig.add_trace(go.Scatter(
        x=future['year'],
        y=future['yhat'],
        name='Baseline Forecast' if idx == 0 else None,
        showlegend=(idx == 0),
        mode='lines+markers',
        line=dict(color='#1f77b4', width=2, dash='dash'),
        marker=dict(size=6, symbol='diamond')
    ), row=row, col=col)
    
    # Tariff-adjusted forecast (the KEY line)
    fig.add_trace(go.Scatter(
        x=future['year'],
        y=future['yhat_tariff_adjusted'],
        name='Tariff-Adjusted Forecast' if idx == 0 else None,
        showlegend=(idx == 0),
        mode='lines+markers',
        line=dict(color='#d62728', width=2, dash='dash'),
        marker=dict(size=6, symbol='x')
    ), row=row, col=col)

    # Add vertical line at 2024 (where forecast begins)
    fig.add_vline(
        x=2024.5, 
        line_dash="dot", 
        line_color="gray",
        row=row, col=col
    )

fig.update_layout(
    title='<b>Trade Export Forecasts: Baseline vs. Tariff-Adjusted (2025-2027)</b><br>'
          '<sup>Red line shows projected impact of current US tariff rates on export growth</sup>',
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()
fig.write_html('../reports/viz4_trade_forecasts.html')
print("✅ Forecast chart saved")

✅ Forecast chart saved
